In [ ]:
import cv2
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import shapely  # library to solve the point in polygoin problem

In [ ]:
%matplotlib notebook

In [ ]:
img = cv2.imread('TMP.png', 0)
img

In [ ]:
plt.figure()
plt.imshow(img)
plt.show()

In [ ]:
# Find contours
# ret, thresh = cv2.threshold(img, 127, 255, 0)
contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [ ]:
draw = np.copy(img)
cv2.drawContours(draw, contours, 1, 127, 20)
cv2.drawContours(draw, contours, 2, 127, 20)

plt.figure()
plt.imshow(draw)
plt.show()

In [ ]:
inner_contour = contours[2]
outer_contour = contours[1]

In [ ]:
inner_contour = contours[2]
outer_contour = contours[1]

x,y = inner_contour.T
x = scipy.signal.decimate(x.astype(np.float32), 10, ftype='iir', n=3)[0]
y = scipy.signal.decimate(y.astype(np.float32), 10, ftype='iir', n=3)[0]
inner_contour = {"x": x, "y": y}


x,y = outer_contour.T
x = scipy.signal.decimate(x.astype(np.float32), 10, ftype='iir', n=3)[0]
y = scipy.signal.decimate(y.astype(np.float32), 10, ftype='iir', n=3)[0]
outer_contour = {"x": x, "y": y}

In [ ]:
plt.figure()

plt.scatter(inner_contour['x'],
            inner_contour['y'],
            s=2)

plt.scatter(outer_contour['x'],
            outer_contour['y'],
            s=2)

plt.axis('scaled')
plt.gca().invert_yaxis()

plt.show()

In [ ]:
inner_contour['dx'] = np.gradient(inner_contour['x'])
inner_contour['dy'] = np.gradient(inner_contour['y'])
outer_contour['dx'] = np.gradient(outer_contour['x'])
outer_contour['dy'] = np.gradient(outer_contour['y'])

# Fix the differences at the tails
inner_contour['dx'][0] = 0.5 * (inner_contour['x'][-1] - inner_contour['x'][1])
inner_contour['dx'][-1] = 0.5 * (inner_contour['x'][-2] - inner_contour['x'][0])
inner_contour['dy'][0] = 0.5 * (inner_contour['y'][-1] - inner_contour['y'][1])
inner_contour['dy'][-1] = 0.5 * (inner_contour['y'][-2] - inner_contour['y'][0])

outer_contour['dx'][0] = 0.5 * (outer_contour['x'][-1] - outer_contour['x'][1])
outer_contour['dx'][-1] = 0.5 * (outer_contour['x'][-2] - outer_contour['x'][0])
outer_contour['dy'][0] = 0.5 * (outer_contour['y'][-1] - outer_contour['y'][1])
outer_contour['dy'][-1] = 0.5 * (outer_contour['y'][-2] - outer_contour['y'][0])

# Calculate the slope
inner_contour['m'] = inner_contour['dy'] / inner_contour['dx']
inner_contour['mp'] = 1 / inner_contour['m']

In [ ]:
fig, ax = plt.subplots(2,2,sharex=True)

n = np.arange(0, len(inner_contour['x']))

ax[0,0].plot(n, inner_contour['x'])
ax[1,0].plot(n, inner_contour['dx'])

ax[0,1].plot(n, inner_contour['y'])
ax[1,1].plot(n, inner_contour['dy'])

plt.show()

plt.figure()
plt.plot(n, inner_contour['mp'])